# Homwork 1

## 1. Stereo Reconstruction and laser-pattern

1.1 Read the color, infrared1, infrared2 images in the folder Homework/HW-1-data (images with numbers (1262, 1755, 1131, 0000))

In [1]:
import cv2
import os

In [2]:
# Read images from the specified directory
def read_images(config):
    images = []
    for suffix in config['prefixes']:
        for num in config['numbers']:
            path = f"{config['root']}/{suffix}{num}.jpg"
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE if 'infra1_' or 'infra2_' in suffix else cv2.IMREAD_COLOR)
            if img is not None:
                images.append(img)
            else:
                print(f"Warning: {path} could not be loaded.")
    return images

1.2 Use OpenCV Stereo Block Matching to find the disparity map, then use the equation for depth to calculate the estimated depth map. You could assume that (focal_length=970 mm, baseline=50 mm) 

In [3]:
def compute_depth_map(infrared1, infrared2, config):
    stereo = cv2.StereoBM_create(config['numDisparities'], config['blockSize'])
    disparity = stereo.compute(infrared1, infrared2)
    depth = (config['focal_length_mm'] * config['baseline_mm']) / (disparity + 1e-6)
    return depth

1.3 Use OpenCV to visualize the reconstructed depth image along with the infrared images using `cv2.imshow`.

In [4]:
config= {
    'root': './HW1-1-data',
    'prefixes': ['infra1_', 'infra2_'],
    'numbers': ['1262', '1755', '1131', '0000'],
    'numDisparities': 0,
    'blockSize': 21,
    'focal_length_mm': 970,
    'baseline_mm': 50,
    'save_path': './outputs'
}

In [6]:
# Create save folder if it doesn't exist
save_folder = config['save_path']
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    print(f"Folder '{save_folder}' created.")

# Compute depth maps
images = read_images(config)
depth_map = [compute_depth_map(images[i],images[i+4],config) for i in range(len(images)//2)]

# Display depth maps
for img,number in zip(depth_map,config['numbers']):
    # file_name = os.path.join(save_folder, f"depth_map_{number}.jpg")
    # cv2.imwrite(file_name, img)
    window_name = f"Depth Map {number}"
    cv2.imshow(window_name, img)

while True:
    k = cv2.waitKey(0)  # 0 makes waitKey wait indefinitely for a key press
    if k != -1:  # If a key was pressed
        print("Key pressed. Exiting...")
        break
cv2.destroyAllWindows()

Key pressed. Exiting...


1.4 What is the difference between the depth quality with respect to 
1. planes with texture (Checkerboard) vs. planes without texture (the PC case)
2. with laser pattern (1262,1755) vs no laser-pattern (0000,1131)

plt.